# Simple Demo of Binary van-Laar solution model
- based on Holland2003

In [ ]:
import numpy as np
import scipy as sp
from scipy import spatial
from scipy import interpolate
import matplotlib.pyplot as plt


In [ ]:
def calc_activity(X, loga_coef):
    activity = X*np.exp(loga_coef)
    return activity

def loga_coef_regular(X, W):
    loga_coef = W*(1-X)**2
    return loga_coef


def loga_coef_vanlaar(X, W, alpha):
    # assumes alpha1=1

    # W_asymm = W*2*alpha1/(alpha1+alpha2)
    # phi1 = X*alpha1/(X*alpha1+(1-X)*alpha2)
    
    W_asymm = W*2*1/(1+alpha)
    phi = X*1/(X*1+(1-X)*alpha)
    loga_coef = W_asymm*(1-phi)**2
    return loga_coef

def calc_XlogX(X):
    XlogX = X*np.log(X)
    XlogX[X==0] = 0
    return XlogX

In [ ]:
X = np.linspace(0,1,301)

In [ ]:
W = 1
alpha = .5

In [ ]:
alpha = .2/(1-.2)
alpha

In [ ]:
loga_coef = loga_coef_regular(X, W)
activity = calc_activity(X, loga_coef)
loga_coef_2 = loga_coef_regular(1-X, W)
activity_2 = calc_activity(1-X, loga_coef_2)


In [ ]:
def calc_regular_soln(X, W):
    loga_coef = loga_coef_regular(X, W)
    activity = calc_activity(X, loga_coef)
    loga_coef_2 = loga_coef_regular(1-X, W)
    activity_2 = calc_activity(1-X, loga_coef_2)

    G_XS = W*X*(1-X)
    S_mix_ideal = -(calc_XlogX(X)+calc_XlogX(1-X))   
    
    model = {}
    model['X'] = X
    model['W'] = W
    model['X1'] = X
    model['X2'] = 1-X
    model['loga_coef1'] = loga_coef
    model['loga_coef2'] = loga_coef_2
    model['a1'] = activity
    model['a2'] = activity_2
    
    model['G_XS'] = G_XS
    model['S_mix_ideal'] = S_mix_ideal
    
    return model

In [ ]:
# G_mix_ideal = -RT*S_mix_ideal
# G_mix = G_mix_ideal + G_XS 

In [ ]:
def calc_asymm_soln(X, W, alpha):
    loga_coef = loga_coef_vanlaar(X, W, alpha)
    activity = calc_activity(X, loga_coef)

    loga_coef_2 = loga_coef_vanlaar(1-X, W, 1/alpha)
    activity_2 = calc_activity(1-X, loga_coef_2)

    G_XS = X*loga_coef + (1-X)*loga_coef_2
    S_mix_ideal = -(calc_XlogX(X)+calc_XlogX(1-X))   
    
    model = {}
    model['X'] = X
    model['W'] = W
    model['X1'] = X
    model['X2'] = 1-X
    model['loga_coef1'] = loga_coef
    model['loga_coef2'] = loga_coef_2
    model['a1'] = activity
    model['a2'] = activity_2
    
    model['G_XS'] = G_XS
    model['S_mix_ideal'] = S_mix_ideal
    
    return model

In [ ]:
mod_regular = calc_regular_soln(X, W)
mod_asymm = calc_asymm_soln(X, W, alpha)

In [ ]:
def plot_aX(mod, label, fmt, figure=None):
    if figure is None:
        figure = plt.figure()

    plt.plot(mod['X'], mod['a1'], fmt,label=label)
    plt.plot(mod['X'], mod['a2'], fmt)

    plt.plot(mod['X'], mod['X'], ':',color='gray')
    plt.plot(mod['X'], 1-mod['X'], ':',color='gray')

    plt.xlim(0,1)
    plt.xlabel('X')
    plt.ylabel('a')
    plt.legend()
    return figure

In [ ]:
figure = plot_aX(mod_regular, 'regular', 'k--')
figure = plot_aX(mod_asymm, 'van laar', 'r-', figure=figure)

# Solvus Calculation

In [ ]:
def get_hull_ind(X, G_mix, fac=None):
    if fac is None:
        fac = 100

    max_G = np.max(np.abs(G_mix))
    
    upper_hull = [0.5, max_G*fac]
    XG = np.vstack((X, G_mix)).T
    XG = np.vstack((XG, upper_hull))
    hull = spatial.ConvexHull(XG)
    
    ind_hull = list(hull.vertices)
    ind_hull.remove(len(X))
    ind_hull = np.sort(ind_hull)

    return ind_hull

def get_solvus_bounds(X, G_mix, fac=None, N=1001, output_hull=False):           
    f_hires = interpolate.interp1d(X, G_mix, kind='cubic')
    X_hires = np.linspace(X[0], X[-1], N)
    G_mix_hires = f_hires(X_hires)
    
    ind_hull = get_hull_ind(X_hires, G_mix_hires, fac)
    
    
    if np.all(~(np.diff(ind_hull)>1)):
        X_gap_bnds = [np.nan, np.nan]
        X_hull = X_hires
        G_mix_hull = G_mix_hires
        
    else:
        ind_gap = np.where(np.diff(ind_hull)>1)[0][0]
        
        ind_gap_bnds = [ind_hull[ind_gap], ind_hull[ind_gap+1]]
        X_gap_bnds = X_hires[ind_gap_bnds]
        
        X_hull = np.hstack((X_hires[:ind_gap_bnds[0]], 
                            X_hires[ind_gap_bnds[1]:]))
        G_mix_hull = np.hstack((G_mix_hires[:ind_gap_bnds[0]], 
                                G_mix_hires[ind_gap_bnds[1]:]))
        
    if output_hull:    
        output = {}
        output['X'] = X_hull
        output['G_mix'] = G_mix_hull
        
        return X_gap_bnds, output
  
    
    return X_gap_bnds

In [ ]:
# ind_hull = get_hull_ind(X, G_mix)

In [ ]:
mod = mod_regular
RT = mod_regular['W']*np.linspace(.1,.55,1001)



In [ ]:
def calc_solvus(RT, mod, N=1001):
    X = mod['X']
    G_XS = mod['G_XS']
    S_mix_ideal = mod['S_mix_ideal']

    X_solvus = []
    for iRT in RT:
        G_mix_ideal = -iRT*S_mix_ideal
        G_mix = G_mix_ideal + G_XS 
    
        X_gap_bnds = get_solvus_bounds(X, G_mix, N=N)
        X_solvus.append(X_gap_bnds)
        
    X_solvus = np.vstack(X_solvus)
    return X_solvus

In [ ]:
def test_solvus_finder(RT_W=.3):
    RT = mod_regular['W']*RT_W
    X = mod['X']
    G_XS = mod['G_XS']
    S_mix_ideal = mod['S_mix_ideal']

    G_mix_ideal = -RT*S_mix_ideal
    G_mix = G_mix_ideal + G_XS 
    X_gap_bnds, hull = get_solvus_bounds(X, G_mix, N=1001, output_hull=True)

    plt.figure()
    plt.plot(X, G_mix, 'k-')
    plt.plot(hull['X'], hull['G_mix'], 'r-')

test_solvus_finder()

In [ ]:
X_solvus = calc_solvus(RT, mod_regular)

In [ ]:
X_solvus_asymm = calc_solvus(RT, mod_asymm)

In [ ]:
plt.figure()
plt.plot(X_solvus[:,0], RT/mod_regular['W'], 'k--', label='regular')
plt.plot(X_solvus[:,1], RT/mod_regular['W'], 'k--')
plt.plot(X_solvus_asymm[:,0], RT/mod_asymm['W'], 'r-', label='van laar')
plt.plot(X_solvus_asymm[:,1], RT/mod_asymm['W'], 'r-')

plt.xlim(-.01,1.01)
plt.legend()

In [ ]:

scale = 900/.5


plt.figure()
plt.plot(X_solvus[:,0], scale*RT/mod_regular['W'], 'k--', label='regular')
plt.plot(X_solvus[:,1], scale*RT/mod_regular['W'], 'k--')
plt.plot(X_solvus_asymm[:,0], scale*RT/mod_asymm['W'], 'r-', label='van laar')
plt.plot(X_solvus_asymm[:,1], scale*RT/mod_asymm['W'], 'r-')


plt.ylim(400,1000)
plt.xlim(-.01,1.01)

plt.legend()
plt.title('Fig 2 (Holland2003)')